In [ ]:
#  Reward function shaping in this script
#  TODO extend for the discrete case also. Right now, discrete uses only sparse rewards

In [ ]:
import pandas as pd
import numpy as np
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle

In [ ]:
df_train = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_set_scaled.csv')
df_val = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_set_scaled.csv')
df_test = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_set_scaled.csv')

In [ ]:
df_train.head()

In [ ]:
len(df_train)

In [ ]:
df_test.head()

In [ ]:
# Smaller rewards to improve stability in continuous models
df_train.loc[df_train['reward'] > 15,'reward'] = 15
df_train.loc[df_train['reward'] < -15,'reward'] = -15

df_val.loc[df_val['reward'] > 15,'reward'] = 15
df_val.loc[df_val['reward'] < -15,'reward'] = -15

df_test.loc[df_test['reward'] > 15,'reward'] = 15
df_test.loc[df_test['reward'] < -15,'reward'] = -15

In [ ]:
# Similar process in discrete models to facilitate comparisons
# df_train_disc.loc[df_train_disc['reward'] > 15,'reward'] = 15
# df_train_disc.loc[df_train_disc['reward'] < -15,'reward'] = -15

# df_val_disc.loc[df_val_disc['reward'] > 15,'reward'] = 15
# df_val_disc.loc[df_val_disc['reward'] < -15,'reward'] = -15

# df_test_disc.loc[df_test_disc['reward'] > 15,'reward'] = 15
# df_test_disc.loc[df_test_disc['reward'] < -15,'reward'] = -15

In [ ]:
orig_df = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\Final2.csv')

In [ ]:
# lactate - target is 0.5 to 1 mmol/l
# SOFA - 0
#  GCS - 0
#  mechvent = 0
# MAP is normally between 65 and 110 mmHg

In [ ]:
# s1 = orig_df['SOFA'].values
# s2 = orig_df['SOFA'].values
# s2 = np.insert(s2,0,9)
# s2 = np.delete(s2, -1)
# print len(s1), len(s2)

In [ ]:
# pd.Series(s2-s1).hist(bins=100)

In [ ]:
# l1 = orig_df['Arterial_lactate'].values
# l2 = orig_df['Arterial_lactate'].values
# l2 = np.insert(l2,0,9)
# l2 = np.delete(l2, -1)
# print len(l1), len(l2)

In [ ]:
c0 = -0.1/4
c1 = -0.5/4
c2 = -2

In [ ]:
# c0 = -0.1/2
# c1 = -0.5/2
# c2 = -2

In [ ]:
# add rewards
orig_df['shaped_reward'] = 0
for i in orig_df.index:
    if i == 0:
        continue
    if orig_df.loc[i, 'stay_id'] == orig_df.loc[i-1, 'stay_id']:
        sofa_cur = orig_df.loc[i,'SOFA']
        sofa_prev = orig_df.loc[i-1,'SOFA']
        lact_cur = orig_df.loc[i,'Arterial_lactate']
        lact_prev = orig_df.loc[i-1,'Arterial_lactate']
        reward = 0
        if sofa_cur == sofa_prev and sofa_cur != 0:
            reward += c0
        reward += c1*(sofa_cur-sofa_prev)
        reward += c2*np.tanh(lact_cur - lact_prev)
        orig_df.loc[i-1,'shaped_reward'] = reward
    if i % 10000 == 0:
        print(i)

# if orig_df.ix[len(orig_df)-1, 'died_in_hosp'] == 1:
#     orig_df.ix[len(orig_df)-1, 'shaped_reward'] = -100
# elif orig_df.ix[len(orig_df)-1, 'died_in_hosp'] == 0:
#      orig_df.ix[len(orig_df)-1, 'shaped_reward'] = 100
print orig_df['shaped_reward'].value_counts()

In [ ]:
%matplotlib inline
orig_df['shaped_reward'].hist(bins=100)

In [ ]:
train_ids = df_train['stay_id'].unique()
val_ids = df_val['stay_id'].unique()
test_ids = df_test['stay_id'].unique()

In [ ]:
train_rewards = orig_df.loc[orig_df['stay_id'].isin(train_ids)]['shaped_reward']

In [ ]:
len(train_rewards)

In [ ]:
train_rewards.tail()

In [ ]:
# check this works as expected
val_rewards = orig_df.loc[orig_df['stay_id'].isin(val_ids)]['shaped_reward']

In [ ]:
test_rewards = orig_df.loc[orig_df['stay_id'].isin(test_ids)]['shaped_reward']

In [ ]:
len(test_rewards)

In [ ]:
len(val_rewards)

In [ ]:
len(orig_df)

In [ ]:
df_train.tail()

In [ ]:
len(orig_df['shaped_reward'])

In [ ]:
len(df_train)

In [ ]:
# check that this sums to the total number of data items -- needs to be re-run
len(df_train) + len(df_test) + len(df_val)

In [ ]:
df_train['reward'] += np.array(train_rewards)
df_val['reward'] += np.array(val_rewards)
df_test['reward'] += np.array(test_rewards)

In [ ]:
# df_train_disc['reward'] += np.array(train_rewards)
# df_val_disc['reward'] += np.array(val_rewards)
# df_test_disc['reward'] += np.array(test_rewards)

In [ ]:
df_train.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_data_final_cont.csv',index=False)
df_val.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_data_final_cont.csv', index=False)
df_test.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_data_final_cont.csv',index=False)

In [ ]:
# df_train_disc.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_data_final_disc.csv',index=False)
# df_val_disc.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_data_final_disc.csv', index=False)
# df_test_disc.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_data_final_disc.csv',index=False)

In [ ]:
train_rewards.head(n=20)

In [ ]:
df_train['reward'].head(n=20)